In [39]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import plotly.graph_objects as go

from scipy.fft import fft
from scipy.signal import butter,filtfilt

import time

# %matplotlib widget

In [40]:
def modifyTimestamps(df):

    df = df.dropna()
    df = df.drop(df.filter(regex='Name..').columns, axis=1)
    df = df.rename(columns={"Name": "Time"})

    return df


def LDAP_to_time(t_array):

    timestamp_array = []

    for timestamp in t_array:

        if type(timestamp) == pd._libs.tslibs.timestamps.Timestamp:
            timestamp_array.append(timestamp)

        else:
            value = datetime.datetime(1601, 1, 1) + datetime.timedelta(seconds=timestamp/10000000)
            timestamp_array.append(value)

    return np.asarray(timestamp_array)

In [41]:
def lowpass_filter(signal, cutoff, fs, order):

    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq

    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, signal)
    
    return y

In [42]:
def rms_variation(V_array, T_array, anomaly_meta_info):

    for i in np.arange(start=0, stop=len(V_array), step=200):

        start = i
        end = i + 400

        V_rms = np.sqrt(np.mean(np.square(V_array[start:end])))


        if (V_rms >= 1.1 * anomaly_meta_info['voltage_threshold']):

            # swell start
            if (anomaly_meta_info['swell_start'] == None) and (anomaly_meta_info['swell_end'] == None):
                anomaly_meta_info['swell_start'] = T_array[i]
                print(f"Swell Start: {anomaly_meta_info['swell_start']}")


            if (V_rms >= anomaly_meta_info['max_swell_value']):
                anomaly_meta_info['max_swell_value'] = V_rms



        if (V_rms < 1.1 * anomaly_meta_info['voltage_threshold']) and (V_rms >= 0.9 * anomaly_meta_info['voltage_threshold']):

            # sag end
            if (anomaly_meta_info['sag_start'] != None) and (anomaly_meta_info['sag_end'] == None):
                anomaly_meta_info['sag_end'] = T_array[i]
                print(f"Sag End: {anomaly_meta_info['sag_end']}")


            # swell end
            if (anomaly_meta_info['swell_start'] != None) and (anomaly_meta_info['swell_end'] == None):
                anomaly_meta_info['swell_end'] = T_array[i]
                print(f"Swell End: {anomaly_meta_info['swell_end']}")


                
        if (V_rms < 0.9 * anomaly_meta_info['voltage_threshold']) and (V_rms >= 0.1 * anomaly_meta_info['voltage_threshold']):

            # sag start
            if (anomaly_meta_info['sag_start'] == None) and (anomaly_meta_info['sag_end'] == None):
                anomaly_meta_info['sag_start'] = T_array[i]
                print(f"Sag Start: {anomaly_meta_info['sag_start']}")


            if (V_rms <= anomaly_meta_info['min_sag_value']):
                anomaly_meta_info['min_sag_value'] = V_rms


            # interruption end
            if (anomaly_meta_info['interruption_start'] != None) and (anomaly_meta_info['interruption_end'] == None):
                anomaly_meta_info['interruption_end'] = T_array[i]
                print(f"Interruption End: {anomaly_meta_info['interruption_end']}")


        
        if (V_rms < 0.1 * anomaly_meta_info['voltage_threshold']):

            # sag end
            if (anomaly_meta_info['sag_start'] != None) and (anomaly_meta_info['sag_end'] == None):
                anomaly_meta_info['sag_end'] = T_array[i]
                print(f"Sag End: {anomaly_meta_info['sag_end']}")


            # interruption start
            if (anomaly_meta_info['interruption_start'] == None) and (anomaly_meta_info['interruption_end'] == None):
                anomaly_meta_info['interruption_start'] = T_array[i]
                print(f"Interruption Start: {anomaly_meta_info['interruption_start']}")



        # reporting detected sag
        if (anomaly_meta_info['sag_start'] != None) and (anomaly_meta_info['sag_end'] != None):
            sag_duration = anomaly_meta_info['sag_end'] - anomaly_meta_info['sag_start']

            print("\n*********************************************************************************")
            print(f"Sag Duration: {sag_duration}")
            print(f"Retained Voltage: {anomaly_meta_info['min_sag_value']}")

            if (sag_duration <= datetime.timedelta(seconds=0.599853)): print(f"Sag Type: Instantaneous")
            elif (sag_duration > datetime.timedelta(seconds=0.599853)) and (sag_duration <= datetime.timedelta(seconds=3)): print(f"Sag Type: Momentary")
            elif (sag_duration > datetime.timedelta(seconds=3)) and (sag_duration <= datetime.timedelta(minutes=1)): print(f"Sag Type: Temporary")
            elif (sag_duration > datetime.timedelta(minutes=1)): print(f"Sag Type: Long Term Undervoltage")
            print("*********************************************************************************")

            anomaly_meta_info['sag_start'] = None
            anomaly_meta_info['sag_end'] = None
            anomaly_meta_info['min_sag_value'] = 1000000.



        # reporting detected swell
        if (anomaly_meta_info['swell_start'] != None) and (anomaly_meta_info['swell_end'] != None):
            swell_duration = anomaly_meta_info['swell_end'] - anomaly_meta_info['swell_start']

            print("\n*********************************************************************************")
            print(f"Swell Duration: {swell_duration}")
            print(f"Retained Voltage: {anomaly_meta_info['max_swell_value']}")

            if (swell_duration <= datetime.timedelta(seconds=0.599853)): print(f"Swell Type: Instantaneous")
            elif (swell_duration > datetime.timedelta(seconds=0.599853)) and (swell_duration <= datetime.timedelta(seconds=3)): print(f"Swell Type: Momentary")
            elif (swell_duration > datetime.timedelta(seconds=3)) and (swell_duration <= datetime.timedelta(minutes=1)): print(f"Swell Type: Temporary")
            elif (swell_duration > datetime.timedelta(minutes=1)): print(f"Swell Type: Long Term Overvoltage")
            print("*********************************************************************************")

            anomaly_meta_info['swell_start'] = None
            anomaly_meta_info['swell_end'] = None
            anomaly_meta_info['max_swell_value'] = 0.



        # reporting detected interruption
        if (anomaly_meta_info['interruption_start'] != None) and (anomaly_meta_info['interruption_end'] != None):
            interruption_duration = anomaly_meta_info['interruption_end'] - anomaly_meta_info['interruption_start']

            print("\n*********************************************************************************")
            print(f"Interruption Duration: {interruption_duration}")

            if (interruption_duration <= datetime.timedelta(seconds=3)): print(f"Interruption Type: Momentary")
            elif (interruption_duration > datetime.timedelta(seconds=3)) and (interruption_duration <= datetime.timedelta(minutes=1)): print(f"Interruption Type: Temporary")
            elif (interruption_duration > datetime.timedelta(minutes=1)): print(f"Interruption Type: Long Term Sustained Interruption")
            print("*********************************************************************************")

            anomaly_meta_info['interruption_start'] = None
            anomaly_meta_info['interruption_end'] = None



    return anomaly_meta_info

In [43]:
def thd_variation(signal, anomaly_meta_info, fs=20000):

    X = fft(signal)
    N = len(X)
    n = np.arange(N)
    T = N/fs


    freq = (n/T)[1:int(N/2)] 
    freq_amplitudes = np.abs(X)[1:int(N/2)]

    sorted_indices = np.argsort(-freq_amplitudes)

    dominant_freq = freq[sorted_indices[0]]
    other_freq = freq[sorted_indices[1:]]

    anomaly_meta_info['dominant_freq'] = dominant_freq


    sq_sum = np.sum(np.square(freq_amplitudes))

    sq_prime = np.square(np.max(freq_amplitudes))
    sq_harmonics = sq_sum - sq_prime

    thd = 100 * np.sqrt(sq_harmonics / sq_prime)

    anomaly_meta_info['thd'] = thd


    # # reporting detected prime harmonic variation
    # if (dominant_freq != anomaly_meta_info['required_freq']): 

    #     print("\n*********************************************************************************")
    #     print(f"Prime Harmonic Variation")
    #     print(f"Dominant Frequency: {anomaly_meta_info['dominant_freq']}")
    #     print("*********************************************************************************")

    
    # # reporting detected THD variation
    # if (thd > anomaly_meta_info['thd_threshold']): 
        
    #     print("\n*********************************************************************************")
    #     print(f"THD Threshold Violated")
    #     print(f"THD: {anomaly_meta_info['thd']} %")
    #     print("*********************************************************************************")


    return anomaly_meta_info

In [44]:
def anomaly_detection(V_array, I_array, T_array, anomaly_meta_info):

    print(f'\n10 cycles received at: {T_array[0]}')


    V1_filt = lowpass_filter(signal=V_array, cutoff=500, fs=20000, order=2)
    I1_filt = lowpass_filter(signal=I_array, cutoff=500, fs=20000, order=2)


    anomaly_meta_info = rms_variation(V_array=V1_filt, T_array=T_array, anomaly_meta_info=anomaly_meta_info)
    anomaly_meta_info = thd_variation(signal=V1_filt, anomaly_meta_info=anomaly_meta_info)


    return anomaly_meta_info

In [45]:
df = pd.read_csv('Power Anomalies Simulations Data/AD3_data/2024_03_12/01_Power Failure.csv', sep=';')
df = modifyTimestamps(df=df)

factor = 192.76315789473685

V1_complete = df['U_L1real'].iloc[:].to_numpy() * factor
I1_complete = df['I_L1real'].iloc[:].to_numpy()
T_complete = df['Time'].iloc[:].to_numpy()


anomaly_meta_info = {
    'voltage_threshold': 230.,

    'swell_start': None,
    'swell_end': None,
    'max_swell_value': 0.,

    'sag_start': None,
    'sag_end': None,
    'min_sag_value': 1000000.,

    'interruption_start': None,
    'interruption_end': None,

    'required_freq': 50.,
    'dominant_freq': None,

    'thd_threshold': 8.,
    'thd': None,
}

In [46]:
for i in np.arange(start=0, stop=len(V1_complete), step=4000):

    start = i
    end = start + 4000

    V1 = V1_complete[start:end]
    I1 = I1_complete[start:end]

    T1 = T_complete[start:end]
    T1 = LDAP_to_time(t_array=T1)

    if len(V1) >= 400:
        anomaly_meta_info = anomaly_detection(V_array=V1, I_array=I1, T_array=T1, anomaly_meta_info=anomaly_meta_info)

    # time.sleep(2)


10 cycles received at: 2024-03-12 11:14:33.108999

10 cycles received at: 2024-03-12 11:14:33.309000

10 cycles received at: 2024-03-12 11:14:33.509001

10 cycles received at: 2024-03-12 11:14:33.709000

10 cycles received at: 2024-03-12 11:14:33.909000

10 cycles received at: 2024-03-12 11:14:34.108999

10 cycles received at: 2024-03-12 11:14:34.309000

10 cycles received at: 2024-03-12 11:14:34.509001

10 cycles received at: 2024-03-12 11:14:34.709000

10 cycles received at: 2024-03-12 11:14:34.909000

10 cycles received at: 2024-03-12 11:14:35.108999

10 cycles received at: 2024-03-12 11:14:35.309000

10 cycles received at: 2024-03-12 11:14:35.509001

10 cycles received at: 2024-03-12 11:14:35.709000

10 cycles received at: 2024-03-12 11:14:35.909000

10 cycles received at: 2024-03-12 11:14:36.108999

10 cycles received at: 2024-03-12 11:14:36.309000

10 cycles received at: 2024-03-12 11:14:36.509001

10 cycles received at: 2024-03-12 11:14:36.709000

10 cycles received at: 2024-03

In [47]:
# fig = plt.figure(figsize=(16, 8))
# plt.plot(V1_complete)

# plt.show()